In [ ]:
! pip install -qq requirements.txt
import torch
import numpy as np
import matplotlib.pyplot as plt
from model import Model
from texts import prepare
import matplotlib as mpl

In [ ]:
opt = Model('125m', limit=1000)
dataset, label, skip = prepare('pile')

In [ ]:
ff_collector = []
attn_collector = []
count = 0

for data in dataset:
    count += 1
    text = data[label]
    #res = opt.get_residual_stream( text ).permute( (1,0,2) ).cpu()
    inpt, ff, attn, output = opt.get_text_activations( text )
    ff   = ff.transpose( 0, 1 )
    attn = attn.transpose( 0, 1 )

    for activation in ff:
        ff_collector.append( activation )
    for activation in attn:
        attn_collector.append( activation )
    
    if count >= 20:
        break

ff_activations = torch.stack( ff_collector ).permute( (1,2,0) )
attn_activations = torch.stack( attn_collector ).permute( (1,2,0) )
print( ff_activations.size() )
print( attn_activations.size() )

In [ ]:
for j in range(opt.n_layers):
    for i in [0, 10, 100, 200, 300, 400, 500 ]:
        counts, bins = np.histogram( ff_activations[j][i].cpu().numpy(), bins=100 )
        mids = (bins[:-1] + bins[1:]) / 2
        plt.semilogy( mids, counts )
    plt.show()

In [ ]:
for j in range(opt.n_layers):
    for i in [0, 10, 100, 200, 300, 400, 500 ]:
        counts, bins = np.histogram( attn_activations[j][i].cpu().numpy(), bins=100 )
        mids = (bins[:-1] + bins[1:]) / 2
        plt.semilogy( mids, counts )
    plt.show()

In [ ]:
attn_means = torch.mean( attn_activations, dim=-1)
attn_indices = torch.sort( attn_means, dim=-1 ).indices.cpu().numpy()
indices = set([])
for i in range(opt.n_layers):
    indices.add( attn_indices[i][-1] )
    indices.add( attn_indices[i][-2] )
indices = np.sort( np.array(list(indices)) )
n_items = len(indices)

In [ ]:
for j in range(12):
    for i in indices:
        rgb = mpl.colors.hsv_to_rgb( (1, 1, i/n_items) )
        print( rgb )
        counts, bins = np.histogram( ff_activations[j][i].cpu().numpy(), bins=50 )
        mids = (bins[:-1] + bins[1:]) / 2
        plt.semilogy( mids, counts, label=f" dim {i}", color = rgb )
    plt.legend()
    plt.show()